In [1]:
import pandas as pd
df = pd.read_csv("Sales.csv", encoding='ISO-8859-1')
df.dtypes

Order Number      int64
Line Item         int64
Order Date       object
Delivery Date    object
CustomerKey       int64
StoreKey          int64
ProductKey        int64
Quantity          int64
Currency Code    object
dtype: object

In [3]:
date_formats = ['%d-%m-%Y', '%m/%d/%Y',]

def parse_dates(date_str):
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT

# Apply the parsing function to the 'date' column
df['Order Date'] = df['Order Date'].apply(parse_dates)
df['Delivery Date'] = df['Delivery Date'].apply(parse_dates)

# Convert to the desired format
df['Order Date'] = df['Order Date'].dt.strftime('%d-%m-%Y')
df['Delivery Date'] = df['Delivery Date'].dt.strftime('%d-%m-%Y')

In [5]:
df.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [7]:
df.dropna(subset=['Delivery Date'], inplace=True)
df.isnull().sum()

Order Number     0
Line Item        0
Order Date       0
Delivery Date    0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency Code    0
dtype: int64

In [9]:
df.tail()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
62878,2243029,6,20-02-2021,27-02-2021,887764,0,1456,2,EUR
62880,2243031,1,20-02-2021,24-02-2021,511229,0,98,4,EUR
62881,2243032,1,20-02-2021,23-02-2021,331277,0,1613,2,CAD
62882,2243032,2,20-02-2021,23-02-2021,331277,0,1717,2,CAD
62883,2243032,3,20-02-2021,23-02-2021,331277,0,464,7,CAD


In [11]:
df.to_csv('Cleaned_Sales.csv', index=False)

In [13]:
df.dtypes

Order Number      int64
Line Item         int64
Order Date       object
Delivery Date    object
CustomerKey       int64
StoreKey          int64
ProductKey        int64
Quantity          int64
Currency Code    object
dtype: object

In [18]:
import pandas as pd
import pymysql
# Connect to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='password', database="project")

# Load CSV data
df = pd.read_csv("Cleaned_Sales.csv")
df = df.dropna()  # Drop null values

# Replace data types from pandas to MySQL
a = ",".join(f"`{i}` {j}" for i, j in zip(df.columns, df.dtypes)).replace("object", "TEXT").replace("int64", "INT")

# Use a valid table name
table_name = "Sales"

# Create the table
create_table_query = f"CREATE TABLE `{table_name}` ({a})"
with myconnection.cursor() as cursor:
    cursor.execute(create_table_query)

# Insert data into the table
for i in range(len(df)):
    insert_query = f"INSERT INTO `{table_name}` VALUES {tuple(df.iloc[i])}"
    with myconnection.cursor() as cursor:
        cursor.execute(insert_query)
    myconnection.commit()

myconnection.close()